# Image Classifier Training

---
## 0. Import Libraries

In [1]:
# import libraries
import os
import caffe
from caffe.proto import caffe_pb2
import lmdb
import numpy as np
import cv2
from sklearn.utils import shuffle

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Flatten, Dense, Activation, Lambda, Cropping2D, Dropout
from keras.layers import Convolution2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using TensorFlow backend.


---
## 1. Load The Data

In [2]:
def load_lmdb(lmdb_data):
    lmdb_env = lmdb.open(lmdb_data)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()
    datum = caffe_pb2.Datum()
    
    X = []
    y = []
    
    for key, value in lmdb_cursor:
        datum.ParseFromString(value)
        label = datum.label
        data = caffe.io.datum_to_array(datum)
    
        #CxHxW to HxWxC in cv2
        image = np.transpose(data, (1,2,0))
        X.append(image)
        y.append(label)
        
    lmdb_env.close()
    return X, y

In [3]:
# Load lmdb data

data_path = "../lmdb/"
train_file = data_path + "train_lmdb/"
valid_file= data_path + "val_lmdb/"
test_file = data_path + "test_lmdb/"

X_train, y_train = load_lmdb(train_file)
X_valid, y_valid = load_lmdb(valid_file)
X_test, y_test = load_lmdb(test_file)
n_classes = 10

----

## Step 2: Design and Test a Model Architecture

### Pre-process the Data Set (normalization, grayscale, etc.)

In [4]:
def preprocess_image(X):
    # Convert to grayscale
    n = np.shape(X)[0]
    gray = []
    for i in range(0, n):
        gray.append(cv2.cvtColor(X[i], cv2.COLOR_RGB2GRAY))
        
    # Normalize the data
    normal = []
    for i in range(0, n):
        one_channel = np.zeros((32, 32, 1), np.float)
        one_channel[:,:,0] = ((gray[i].astype(np.float) - 128) / 128) 
        normal.append(one_channel)
        
    return normal

In [5]:
X_train = np.array(preprocess_image(X_train))
X_valid = np.array(preprocess_image(X_valid))
X_test = np.array(preprocess_image(X_test))
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test)

### Define your architecture here.

In [6]:
np.shape(X_train)

(6003, 32, 32, 1)

In [7]:
### Define your architecture here.
model = Sequential()
# model.add(Convolution2D(6,5,5,subsample=(1,1),activation="relu",input_shape=(32,32,1)))
model.add(Conv2D(6, (5, 5), activation="relu", input_shape=(32, 32, 1), strides=(1, 1)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
# model.add(Convolution2D(16,5,5,subsample=(1,1),activation="relu"))
model.add(Conv2D(16, (5, 5), activation="relu", strides=(1, 1)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## Train the Model
Run the training data through the training pipeline to train the model.
Before each epoch, shuffle the training set.
After each epoch, measure the loss and accuracy of the validation set.
Save the model after training.

In [8]:
print("Fit model on training data")
history=model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data=(X_valid, y_valid))
model.save('model.h5')

Fit model on training data

Train on 6003 samples, validate on 2062 samples
Epoch 1/20
6003/6003 [==============================] - 4s 633us/step - loss: 1.3775 - val_loss: 0.4818
Epoch 2/20
6003/6003 [==============================] - 4s 693us/step - loss: 0.4394 - val_loss: 0.2603
Epoch 3/20
6003/6003 [==============================] - 5s 881us/step - loss: 0.2703 - val_loss: 0.1917
Epoch 4/20
6003/6003 [==============================] - 4s 692us/step - loss: 0.1787 - val_loss: 0.1533
Epoch 5/20
6003/6003 [==============================] - 4s 668us/step - loss: 0.1393 - val_loss: 0.1249
Epoch 6/20
6003/6003 [==============================] - 4s 678us/step - loss: 0.1075 - val_loss: 0.1199
Epoch 7/20
6003/6003 [==============================] - 5s 769us/step - loss: 0.0830 - val_loss: 0.1172
Epoch 8/20
6003/6003 [==============================] - 4s 738us/step - loss: 0.0746 - val_loss: 0.0991
Epoch 9/20
6003/6003 [==============================] - 4s 687us/step - loss: 0.0628 - val_l

## Evaluate the Model
Once you are completely satisfied with your model, evaluate the performance of the model on the test set.
Be sure to only do this once!
If you were to measure the performance of your trained model on the test set, then improve your model, and then measure the performance of your model on the test set again, that would invalidate your test results. You wouldn't get a true measure of how well your model would perform against real data.

In [9]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
1935/1935 [==============================] - 0s 237us/step
test loss, test acc: 0.08067338933827979
Generate predictions for 3 samples
predictions shape: (3, 10)
